# Session 14: Assembly techniques (with more accuracy)
Made by:

**- Ruelas Flores, César Diego**

This is a new notebook, 'LAB14-RUELAS-robust.ipynb'. We will focus on three pillars to maximize performance:

Advanced Feature Engineering: We will extract valuable information from the date column and create new features to give the model greater predictive power.
State-of-the-art Models: We will incorporate XGBoost and LightGBM, the industry-standard models for tabular data.
Robust Optimization and Evaluation: We will use RandomizedSearchCV with cross-validation to efficiently find the best hyperparameters and obtain highly reliable performance estimates.

In [1]:
# %pip install xgboost lightgbm catboost -q (esto va a demorar como unos 5 minutos o menos)

In [2]:
# --- Configuración General del Entorno ---
import os
import sys
import logging
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --- Modelos Avanzados ---
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

# --- Configuración Centralizada del Logging ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)

logging.info("Entorno ROBUSTO configurado y librerías importadas.")

2025-06-19 17:40:49,072 - root - INFO - Entorno ROBUSTO configurado y librerías importadas.


## 1. VARIABLES GLOBALES

In [3]:
TARGET_COLUMN = 'actual_productivity'
TEST_SIZE = 0.2
RANDOM_STATE = 42
# Número de iteraciones para la búsqueda aleatoria de hiperparámetros
N_ITER_SEARCH = 20

logging.info("Variables globales definidas.")

2025-06-19 17:40:49,085 - root - INFO - Variables globales definidas.


## 2. FUNCIONES (../src/utils_robust.py)

In [4]:
%%writefile ../src/utils_robust.py
import logging
from typing import List, Tuple, Dict

import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo

from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
# --- IMPORTACIÓN AÑADIDA ---
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.compose import ColumnTransformer

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

# Obtenemos el logger para este módulo
logger = logging.getLogger(__name__)

def load_and_engineer_features() -> pd.DataFrame:
    """
    Carga los datos y realiza ingeniería de características avanzada.
    """
    logger.info("Iniciando carga de datos e ingeniería de características...")
    try:
        garment_prod = fetch_ucirepo(id=597)
        df = pd.concat([garment_prod.data.features, garment_prod.data.targets], axis=1)
        
        df['date'] = pd.to_datetime(df['date'])
        df['month'] = df['date'].dt.month
        df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
        df['day_of_week'] = df['date'].dt.dayofweek
        df = df.drop(columns=['date'])
        logger.info("Características de tiempo creadas desde la columna 'date'.")

        df['incentive_per_target'] = df['incentive'] / (df['targeted_productivity'] + 1e-6)
        df['smv_per_worker'] = df['smv'] / (df['no_of_workers'] + 1e-6)
        logger.info("Características de ratio creadas.")

        for col in df.columns[df.isnull().any()]:
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(df[col].median())
        logger.info("Valores nulos imputados con la mediana.")
                
        logger.info(f"Proceso finalizado. Dimensiones del DataFrame: {df.shape}")
        return df
    except Exception as e:
        logger.error(f"Error durante la carga e ingeniería de características: {e}")
        raise

def find_best_model(X_train: pd.DataFrame, y_train: pd.Series, n_iter: int, cv: int, random_state: int) -> Tuple[object, Dict]:
    """
    Busca el mejor modelo y sus hiperparámetros usando RandomizedSearchCV.
    """
    logger.info("Iniciando la búsqueda del mejor modelo robusto...")

    numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

    # --- CAMBIO EN EL PREPROCESADOR ---
    # Se reemplaza 'passthrough' por OneHotEncoder para las variables categóricas.
    # handle_unknown='ignore' evita errores durante la validación cruzada si una categoría
    # no está presente en algún pliegue (fold).
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ],
        remainder='passthrough'
    )
    
    search_spaces = {
        'RandomForest': (
            RandomForestRegressor(random_state=random_state),
            {
                'model__n_estimators': [100, 200, 300],
                'model__max_depth': [10, 20, 30],
                'model__min_samples_leaf': [1, 2, 4],
                'model__max_features': ['sqrt', 'log2'],
            }
        ),
        'XGBoost': (
            XGBRegressor(random_state=random_state, objective='reg:squarederror'),
            {
                'model__n_estimators': [100, 200, 500],
                'model__learning_rate': [0.01, 0.05, 0.1],
                'model__max_depth': [3, 5, 7],
                'model__subsample': [0.7, 0.8, 0.9],
                'model__colsample_bytree': [0.7, 0.8, 0.9],
            }
        ),
        'LightGBM': (
            LGBMRegressor(random_state=random_state),
            {
                'model__n_estimators': [100, 200, 500],
                'model__learning_rate': [0.01, 0.05, 0.1],
                'model__num_leaves': [20, 31, 40],
                'model__max_depth': [-1, 10, 20],
                'model__subsample': [0.7, 0.8, 0.9],
            }
        )
    }

    best_model = None
    best_score = -np.inf
    results = {}

    for name, (model, params) in search_spaces.items():
        logger.info(f"--- Optimizando {name} ---")
        
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

        search = RandomizedSearchCV(
            pipeline, params, n_iter=n_iter, scoring='r2', n_jobs=-1,
            cv=cv, verbose=1, random_state=random_state
        )
        search.fit(X_train, y_train)

        logger.info(f"Mejor R2 score para {name} (CV): {search.best_score_:.4f}")
        logger.info(f"Mejores parámetros: {search.best_params_}")

        results[name] = {'best_score': search.best_score_, 'best_params': search.best_params_}

        if search.best_score_ > best_score:
            best_score = search.best_score_
            best_model = search.best_estimator_
            results['best_overall_model'] = name
    
    logger.info(f"Búsqueda finalizada. Mejor modelo global: {results['best_overall_model']}")
    return best_model, results

Overwriting ../src/utils_robust.py


## 3. IMPLEMENTACIÓN PRINCIPAL

In [5]:
# --- Configuración de la ruta del proyecto ---
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    # Usamos un alias para mantener el código limpio
    from src import utils_robust as utils
    logging.info("Módulo 'utils_robust' importado exitosamente.")
except Exception as e:
    logging.error(f"Error en la configuración inicial del path: {e}")

2025-06-19 17:40:49,134 - root - INFO - Módulo 'utils_robust' importado exitosamente.


### 3.1 Parte A: Preprocesamiento con Ingeniería de Características

In [6]:
logging.info("--- [PARTE A] Iniciando preprocesamiento robusto ---")
# La función ahora se encarga de la carga, limpieza e ing. de características.
df_engineered = utils.load_and_engineer_features()
logging.info("--- [PARTE A] Preprocesamiento robusto finalizado ---")

2025-06-19 17:40:49,147 - root - INFO - --- [PARTE A] Iniciando preprocesamiento robusto ---
2025-06-19 17:40:49,149 - src.utils_robust - INFO - Iniciando carga de datos e ingeniería de características...
2025-06-19 17:40:50,987 - src.utils_robust - INFO - Características de tiempo creadas desde la columna 'date'.
2025-06-19 17:40:50,990 - src.utils_robust - INFO - Características de ratio creadas.
2025-06-19 17:40:50,993 - src.utils_robust - INFO - Valores nulos imputados con la mediana.
2025-06-19 17:40:50,994 - src.utils_robust - INFO - Proceso finalizado. Dimensiones del DataFrame: (1197, 19)
2025-06-19 17:40:50,995 - root - INFO - --- [PARTE A] Preprocesamiento robusto finalizado ---


### 3.2. Parte B: Separación de Datos

In [7]:
logging.info("--- [PARTE B] Iniciando separación de datos ---")
# Separar características (X) y objetivo (y) antes de cualquier transformación
X = df_engineered.drop(columns=TARGET_COLUMN)
y = df_engineered[TARGET_COLUMN]

2025-06-19 17:40:51,006 - root - INFO - --- [PARTE B] Iniciando separación de datos ---


In [8]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
logging.info(f"Datos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")
logging.info("--- [PARTE B] Separación de datos finalizada ---")

2025-06-19 17:40:51,029 - root - INFO - Datos divididos: 957 para entrenamiento, 240 para prueba.
2025-06-19 17:40:51,031 - root - INFO - --- [PARTE B] Separación de datos finalizada ---


### 3.3. Parte C: Búsqueda, Entrenamiento y Evaluación del Mejor Modelo

In [9]:
logging.info("--- [PARTE C] Iniciando búsqueda y entrenamiento del mejor modelo ---")
# La función ahora encuentra y entrena el mejor modelo usando CV y búsqueda de hiperparámetros.
# El escalado y la codificación se realizan DENTRO del pipeline en la función.
best_model_pipeline, training_results = utils.find_best_model(
    X_train, y_train, n_iter=N_ITER_SEARCH, cv=5, random_state=RANDOM_STATE
)
logging.info("--- [PARTE C] Búsqueda y entrenamiento finalizados ---")

2025-06-19 17:40:51,045 - root - INFO - --- [PARTE C] Iniciando búsqueda y entrenamiento del mejor modelo ---
2025-06-19 17:40:51,047 - src.utils_robust - INFO - Iniciando la búsqueda del mejor modelo robusto...
2025-06-19 17:40:51,051 - src.utils_robust - INFO - --- Optimizando RandomForest ---
Fitting 5 folds for each of 20 candidates, totalling 100 fits
2025-06-19 17:41:06,845 - src.utils_robust - INFO - Mejor R2 score para RandomForest (CV): 0.4900
2025-06-19 17:41:06,847 - src.utils_robust - INFO - Mejores parámetros: {'model__n_estimators': 300, 'model__min_samples_leaf': 2, 'model__max_features': 'sqrt', 'model__max_depth': 10}
2025-06-19 17:41:06,847 - src.utils_robust - INFO - --- Optimizando XGBoost ---
Fitting 5 folds for each of 20 candidates, totalling 100 fits
2025-06-19 17:41:13,214 - src.utils_robust - INFO - Mejor R2 score para XGBoost (CV): 0.4993
2025-06-19 17:41:13,216 - src.utils_robust - INFO - Mejores parámetros: {'model__subsample': 0.9, 'model__n_estimators': 2

### 3.4. Evaluación de resultados

#### 3.4.1. Evaluación Final del Mejor Modelo en el Conjunto de Prueba

In [10]:
logging.info("--- EVALUACIÓN FINAL SOBRE DATOS DE PRUEBA (TEST SET) ---")
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

2025-06-19 17:41:40,098 - root - INFO - --- EVALUACIÓN FINAL SOBRE DATOS DE PRUEBA (TEST SET) ---


In [11]:
# Usar el pipeline del mejor modelo para predecir en el conjunto de prueba
y_pred_final = best_model_pipeline.predict(X_test)

In [12]:
# Calcular métricas finales
final_r2 = r2_score(y_test, y_pred_final)
final_rmse = np.sqrt(mean_squared_error(y_test, y_pred_final))
final_mae = mean_absolute_error(y_test, y_pred_final)

In [ ]:
# Imprimir resultados del entrenamiento y la evaluación final
print("\n" + "="*50)
logging.info("Resultados de la Búsqueda y Optimización (sobre datos de entrenamiento con CV):")
for model_name, result in training_results.items():
    if model_name != 'best_overall_model':
        logging.info(f"  - {model_name}: Mejor R2 (CV) = {result['best_score']:.4f}")

print("\n" + "="*50)
logging.info(f"Rendimiento del Mejor Modelo ('{training_results['best_overall_model']}') en el Conjunto de Prueba (Test Set):")
logging.info(f"  - R2 Score Final: {final_r2:.4f}")
logging.info(f"  - RMSE Final:   {final_rmse:.4f}")
logging.info(f"  - MAE Final:    {final_mae:.4f}")
print("="*50)

logging.info(f"""
--- CONCLUSIÓN ROBUSTA ---
Tras una exhaustiva ingeniería de características y optimización de hiperparámetros,
el mejor modelo encontrado fue '{training_results['best_overall_model']}'.
En el conjunto de datos de prueba, que el modelo nunca había visto, se obtuvo un
R² Score de {final_r2:.4f}. Este resultado representa el rendimiento real y
generalizable del modelo final.
""")


2025-06-19 17:41:40,162 - root - INFO - Resultados de la Búsqueda y Optimización (sobre datos de entrenamiento con CV):
2025-06-19 17:41:40,164 - root - INFO -   - RandomForest: Mejor R2 (CV) = 0.4900
2025-06-19 17:41:40,165 - root - INFO -   - XGBoost: Mejor R2 (CV) = 0.4993
2025-06-19 17:41:40,166 - root - INFO -   - LightGBM: Mejor R2 (CV) = 0.4605

2025-06-19 17:41:40,168 - root - INFO - Rendimiento del Mejor Modelo ('XGBoost') en el Conjunto de Prueba (Test Set):
2025-06-19 17:41:40,169 - root - INFO -   - R2 Score Final: 0.4819
2025-06-19 17:41:40,170 - root - INFO -   - RMSE Final:   0.1173
2025-06-19 17:41:40,172 - root - INFO -   - MAE Final:    0.0768
2025-06-19 17:41:40,173 - root - INFO - 
--- CONCLUSIÓN ROBUSTA ---
Tras una exhaustiva ingeniería de características y optimización de hiperparámetros,
el mejor modelo encontrado fue 'XGBoost'.
En el conjunto de datos de prueba, que el modelo nunca había visto, se obtuvo un
R² Score de 0.4819. Este resultado representa el rend

***

Aunque a primera vista el R² de 0.4973 del modelo 'Bagging' (de nuestro LAB14-RUELAS.ipynb) parece más alto, la clave para determinar cuál es "mejor" no está solo en el número final, sino en la **confianza, robustez y fiabilidad** del proceso que lo generó.

Aquí la comparación detallada:

| Característica | Resultado 1 (Simple) | Resultado 2 (Robusto) | Ganador |
| :--- | :--- | :--- | :--- |
| **Mejor Modelo** | Bagging | XGBoost | **XGBoost** |
| **R² Score Final** | 0.4973 (en Test Set) | 0.4819 (en Test Set) | (Número más alto: Bagging) |
| **RMSE Final** | 0.1155 (en Test Set) | 0.1173 (en Test Set) | (Error más bajo: Bagging) |
| **Metodología** | Evaluación simple (un solo split) | Búsqueda exhaustiva, Ingeniería de Características y Validación Cruzada (CV) | **XGBoost** |
| **Confianza** | Baja (podría ser un resultado por "suerte") | Alta (rendimiento probado y generalizable) | **XGBoost** |

#### Justificación Detallada

1.  **Fiabilidad del Proceso:** El resultado del modelo Bagging proviene de una única división de datos. Esto significa que el R² de 0.4973 podría ser producto del azar; simplemente la división de datos de entrenamiento y prueba resultó ser favorable para ese modelo en particular. No tenemos garantía de que se comporte igual con una división de datos diferente.

2.  **El Poder de la Validación Cruzada (CV):** El proceso del modelo XGBoost es mucho más riguroso. Antes de llegar al resultado final, se realizó una "Búsqueda y Optimización" usando validación cruzada. El resultado de `XGBoost: Mejor R2 (CV) = 0.4993` nos dice que, en promedio, a través de múltiples divisiones de los datos de entrenamiento, el modelo tuvo un excelente rendimiento.

3.  **Confirmación con el Test Set:** El paso final fue tomar el mejor modelo de la fase de CV (XGBoost) y evaluarlo en el conjunto de prueba, que nunca antes había visto. El resultado fue un `R2 Score Final: 0.4819`. Que este número sea muy cercano al resultado de la validación cruzada (0.4993 vs 0.4819) es una **excelente señal**. Confirma que el modelo no está sobreajustado y que su rendimiento es estable y predecible.

**En resumen:**

El resultado del modelo **Bagging** es como una buena foto instantánea: podría ser favorecedora, pero no cuenta toda la historia.

El resultado del modelo **XGBoost** es como un video bien grabado: nos muestra un rendimiento consistente y fiable a lo largo del tiempo. Aunque el número final sea una fracción más bajo, es un resultado en el que **puedes confiar** para hacer predicciones sobre datos futuros. En cualquier proyecto profesional de ciencia de datos, la confianza y la generalización del modelo son mucho más valiosas que un número ligeramente más alto obtenido de un proceso menos riguroso.

## 4. TESTING (../tests/test_utils_robust.py)

In [14]:
%%writefile ../tests/test_utils_robust.py
import pytest
import pandas as pd
import numpy as np
import sys
import os

project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src import utils_robust

def test_load_and_engineer_features():
    """
    Prueba que la carga de datos y la ingeniería de características funcionen.
    """
    df = utils_robust.load_and_engineer_features()
    
    # 1. Comprobar que es un DataFrame de pandas
    assert isinstance(df, pd.DataFrame)
    
    # 2. Comprobar que 'date' fue eliminada y las nuevas columnas de tiempo existen
    assert 'date' not in df.columns
    assert 'month' in df.columns
    assert 'week_of_year' in df.columns
    assert 'day_of_week' in df.columns
    
    # 3. Comprobar que las nuevas características de ratio existen
    assert 'incentive_per_target' in df.columns
    assert 'smv_per_worker' in df.columns
    
    # 4. Comprobar que no hay valores nulos
    assert df.isnull().sum().sum() == 0

Overwriting ../tests/test_utils_robust.py


## 5. EJECUCIÓN

# 5.1. Ejecución de Pruebas Unitarias

In [15]:
import sys
# Esta es la ruta del Python que usa tu KERNEL (debería estar dentro de .venv)
print(f"Python del Kernel Jupyter: {sys.executable}")

Python del Kernel Jupyter: c:\Users\AzShet\Documents\Jupyter_LAB\jupyter_projects\5to_ciclo\DataMining\lab14\.venv\Scripts\python.exe


In [16]:
!pytest ../tests/test_utils_robust.py -v -p no:cacheprovider

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.4.1, pluggy-1.6.0 -- C:\Users\AzShet\Documents\Jupyter_LAB\jupyter_projects\5to_ciclo\DataMining\lab14\.venv\Scripts\python.exe
rootdir: c:\Users\AzShet\Documents\Jupyter_LAB\jupyter_projects\5to_ciclo\DataMining\lab14\Data_Mining-LAB14
collecting ... collected 1 item

..\tests\test_utils_robust.py::test_load_and_engineer_features PASSED    [100%]

============================== 1 passed in 4.61s ==============================
